# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

[2023-01-09 15:08:07,644][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-01-09 15:08:07,696][INFO]: Connected tobiasr@172.26.128.53:3306


DataJoint connection (connected) tobiasr@172.26.128.53:3306

Manual entry:

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

### Imports and activation

Importing schema from `adamacs.pipeline` automatically activates relevant schema.

In [2]:
import datajoint as dj
from adamacs.pipeline import subject, session, surgery, scan, event, trial
from adamacs import utility
from adamacs.ingest import behavior as ibe

Loading DLC 2.2.1.1...


/Users/trose/opt/miniconda3/envs/datajoint/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


Assign easy names for relevant tables

In [3]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())

In [4]:
event_types = ['main_track_gate', 'shutter', 'mini2p_frames', 'mini2p_lines', 'mini2p_volumes', 'aux_bpod_cam',
               'aux_bpod_visual', 'aux_bpod_reward', 'aux_bpod_tone']

for e in event_types:
    event.EventType.insert1({'event_type': e, 'event_type_description': ''}, skip_duplicates=True,)

In [4]:
session.Session()

session_id,subject PyRat import uses this for earmark value,session_datetime
sess9FF6U114,ROS-1438,2022-11-22 00:00:00


## Ingesting behavior

In [6]:
ibe.ingest_aux("sess9FF6U114",verbose=True)

Number of scans found: 1


/Users/trose/opt/miniconda3/envs/datajoint/lib/python3.8/site-packages/pywavesurfer/ws.py:46: RuntimeWarning: You are reading a WaveSurfer file version this module was not tested with: file version 1.0.5, latest version tested: 0.97
  warnings.warn('You are reading a WaveSurfer file version this module was not tested with: '


In [5]:
event.Event()

session_id,event_type,event_start_time (second) relative to recording start,event_end_time (second) relative to recording start
sess9FF6U114,aux_bpod_cam,0.0,0.005
sess9FF6U114,aux_bpod_cam,0.01665,0.02165
sess9FF6U114,aux_bpod_cam,0.03335,0.03835
sess9FF6U114,aux_bpod_cam,0.05,0.055
sess9FF6U114,aux_bpod_cam,0.06665,0.07165
sess9FF6U114,aux_bpod_cam,0.08335,0.08835
sess9FF6U114,aux_bpod_cam,0.1,0.105
sess9FF6U114,aux_bpod_cam,0.11665,0.12165
sess9FF6U114,aux_bpod_cam,0.13335,0.13835
sess9FF6U114,aux_bpod_cam,0.15,0.155


In [6]:
event.EventType()

event_type,event_type_description
aux_bpod_cam,
aux_bpod_reward,
aux_bpod_tone,
aux_bpod_visual,
main_track_gate,
mini2p_frames,
mini2p_lines,
mini2p_volumes,
shutter,


In [7]:
event.BehaviorRecording()

session_id,recording_start_time,recording_duration,recording_notes
sess9FF6U114,2022-11-22 16:13:20,600.0,


In [9]:
dj.Diagram(event)

TypeError: argument of type 'bool' is not iterable

In [12]:
session.Session * session.SessionDirectory

session_id,subject PyRat import uses this for earmark value,session_datetime,session_dir Path to the data directory for a session,user_id
sess9FF6U114,ROS-1438,2022-11-22 00:00:00,/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6U114_sess9FF6U114,6


In [13]:
key='scan9FF6U114'
(scan.Scan & f'scan_id=\"{key}\"')

session_id,scan_id,scanner,acq_software,scan_notes free-notes
sess9FF6U114,scan9FF6U114,mini2p1,ScanImage,


In [14]:
scan.Scan()

session_id,scan_id,scanner,acq_software,scan_notes free-notes
sess9FF6U114,scan9FF6U114,mini2p1,ScanImage,


In [15]:
scan.ScanInfo.populate()

`CB DEV NOTE:` In demo file, there are no `scan.motor_position_at_zero` values. Set to 0 here to avoid downstream type errors when adding. Future would should set these to null when not present and revise downstream code.

In [16]:
scan.ScanInfo()

session_id,scan_id,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage),scan_datetime datetime of the scan,scan_duration (seconds) duration of the scan
sess9FF6U114,scan9FF6U114,1,1,1,9250,0,nan,nan,nan,15.3845,1,250.003,0.95,None,601.255


In [17]:
scan.ScanInfo.Field()

session_id,scan_id,field_idx,px_height height in pixels,px_width width in pixels,um_height height in microns,um_width width in microns,field_x (um) center of field in the motor coordinate system,field_y (um) center of field in the motor coordinate system,field_z (um) relative depth of field,delay_image (ms) delay between the start of the scan and pixels in this field,roi the scanning roi (as recorded in the acquisition software) containing this field - only relevant to mesoscale scans
sess9FF6U114,scan9FF6U114,0,256,256,65.7966,65.7966,nan,nan,nan,=BLOB=,None


Note the relative path below:

In [18]:
scan.ScanInfo.ScanFile()

session_id,scan_id,file_path filepath relative to root data directory
sess9FF6U114,scan9FF6U114,/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6U114_sess9FF6U114/scan9FF6U114_JJ_ROS_1438_00004.tif
